# Inleiding
Deze notebook bestaat om mijn proces te laten zien hoe ik tot de volgende researchstory afhandel.
- RS6.1 ALS team WILLEN WIJ weten hoe het door de opdrachtgever aangeleverde CSV bestand inkunnen lezen en per categorie kunnen tellen

De start is voor mij pittig omdat ik nog niks tot weinig van de benodigde packages afweet waaronder numpy, pandas en matlab
Verder is het onhandig dat ik nog niet goed wist hoe ik het proces het beste kon documenteren. Ik heb dus achteraf veel documentatie in deze jupyter notebook geschreven hierdoor is met name het codeer stuk niet helemaal goed vastgelegd.

## Voorbereiding
ter voorbereiding heb ik advies gevraagd aan nik aangezien die dit deel van de opleiding eerder gevolgd heeft nik gaf aan dat een csv bestand door pandas ingelezen kon worden en daarna automatisch geteld kon worden. Verder gaf die mij de suggestie dat ik de categorien uiteindelijk beter kon uitspreiden in aparte colommen met boolean waarden per rij omdat een ML algoritme hierop beter getraint kan worden. 

Waar ik me ook zorgen over maakte was hoe ik echte analyses uit de data kan halen. Hierop heb ik de AI docent Henry Maathuis om advies gevraagd. Henry maathuis vertelde me dat ik eerst de data goed moest schoon maken van type foutjes en en inconsistenties. Tellen en visualiseren om voor jezelf in kaart te brengen wat is nou waardevolle data. om vervolgens de data klaar te maken voor machine learning door kolommen met velden die meerdere waarden hebben uit te spreiden over 1 waarde per kolom.

## panda installatie
Tijdens het coderen heb ik eerst uitgezocht hoe ik een csv naar een pandas dataframe moest omzetten.
Hiervoor moest ik eerst pandas installeren. Tijdens het installeren bleek er een bug in mijn python installatie te zitten die ervoor zorgde dat de installatie met pip niet werkte. Dit heb ik opelost met behulp van prompting in chatgbt - de chat is hier te vinden https://chatgpt.com/share/68d40af3-abdc-8000-82fd-de0bc4d15325 Hierna heb ik pandas gewoon kunnen installeren.

In [50]:
#setup
import pandas as pd
repoPath = 'C:/Users/Tommy/Documents/GitHub/retraction-watch/' # dit dient iemand zelf correct in te stellen.
dataPath = repoPath + 'armand/data/'
sourcePath = dataPath + 'source.csv'

## inlezen en printen van een csv
Op dit moment heb ik vooral uitgezocht hoe ik onze data kon inlezen en printen. Ik heb voor het ophalen een functie eromheen gezet om deze naar een volgend project mee te kunnen nemen.

In [51]:
def get_csv(source, index, array):
	df = pd.read_csv(source, index_col=index, usecols=array)
	return df

# data source wordt los meegeleverd en niet in de repo gedeeld uit vertrouwlijkheids overwegingen.
df = get_csv(sourcePath, 'Record ID', ['Record ID', 'Title', 'Subject', 'Institution', 'Journal', 'Publisher', 'Country', 'Author', 'ArticleType', 'RetractionDate', 'OriginalPaperDate', 'RetractionNature', 'Reason', 'Paywalled' ])
print(df)

                                                       Title  \
Record ID                                                      
66854      High-Fat Diet Promotes DSS-Induced Ulcerative ...   
66852      Fuzzy Logic-Based Systems for the Diagnosis of...   
66851      Flat Panel CT Scanning Is Helpful in Predictin...   
66850      Mangifera indica Extracts as Novel PKM2 Inhibi...   
66849      Mechanism of Radix Astragali and Radix Salviae...   
...                                                      ...   
5          Effect of Perindopril on Large Artery Stiffnes...   
4          MtvR is a global small noncoding regulatory RN...   
3          The second RNA chaperone, Hfq2, is also requir...   
2          Regulation of Wnt/beta-catenin pathway by cPLA...   
1          Genistein protects genioglossus myocyte agains...   

                                                     Subject  \
Record ID                                                      
66854      (BLS) Genetics;(HSC) Medicin

# tellen
Hierna ben ik de subjects gaan tellen. Echter is door het mixen van de verschillende waarden dt niet betrouwbaar omdat deze in combinatie als een waarde gezien worden. Zoals in onderstaand voorbeeld.

In [52]:
def count(df, colName):
	return df[colName].value_counts()

subject_df = get_csv(sourcePath, 'Record ID', ['Record ID', 'Subject'])
out = count(subject_df, "Subject")
print(out)

Subject
(BLS) Biology - Cancer;(BLS) Biology - Cellular;(BLS) Genetics;                                                                                                    1492
(B/T) Computer Science;(B/T) Technology;                                                                                                                           1169
(B/T) Computer Science;                                                                                                                                             793
(BLS) Biochemistry;(BLS) Biology - Cancer;(BLS) Biology - Cellular;(BLS) Genetics;                                                                                  744
(PHY) Mathematics;                                                                                                                                                  614
                                                                                                                                                        

## Custom telcode
Na het vorige ben ik zelf een stukje code gaan schrijven om door de dataframe per rij heen te loopen en de kolommen te splitsen.
Na dit splitsen wou ik dit per rij in volledigheid geprint hebben en als csv op kunnen slaan. Hier heb ik 2 aparte functies voor gemaakt.


In [53]:
def save_csv(df, path):
	df.to_csv(path, index=True)

def get_unique_subjects(df):
	subjects = {}

	for i in range(len(df)):
		rowSubjects = df.iloc[i]['Subject'].split(';')

		for s in rowSubjects:
			if s in subjects.keys():
				subjects[s] += 1
			elif s != '':
				subjects[s] = 1		

	out = {
		"count": [],
		"subject": []
	}
	index = []

	id = 0
	for key in subjects.keys():
		index.append(id)
		out["count"].append(subjects[key])
		out["subject"].append(key)
		id += 1
	out = pd.DataFrame(out, index=index)
	out.index.name = "ID"
	return out

def print_dataframe(df, max=float('inf')):
	headline = ""
	if df.index.name == "":
		headline += "index"
	else:
		headline += f"{df.index.name}"

	for k in df.keys():
		headline += f", {k}"
		
	print(headline)


	index = 0
	for item in df.itertuples():
		if (max < index ):
			break
		line = ""
		for i in range(len(item)):
			if i == 0:
				line += f"{item[i]}"
			else:
				line += f", {item[i]}"
		index+= 1
		
		print(f"{line}")

def saveUniqueSubjects():
	df = get_csv(sourcePath, 'Record ID', ['Record ID', 'Title', 'Subject', 'Institution', 'Journal', 'Publisher', 'Country', 'Author', 'ArticleType', 'RetractionDate', 'OriginalPaperDate', 'RetractionNature', 'Reason', 'Paywalled' ])
	data = get_unique_subjects(df)
	save_csv(data, dataPath + "subjects.csv")
	print_dataframe(data)

saveUniqueSubjects()

ID, count, subject
0, 8352, (BLS) Genetics
1, 1773, (HSC) Medicine - Gastroenterology
2, 276, (HSC) Nutrition
3, 11944, (B/T) Technology
4, 931, (HSC) Medicine - Urology/Nephrology
5, 2442, (HSC) Medicine - Cardiovascular
6, 2350, (HSC) Medicine - Neurology
7, 1571, (HSC) Radiology/Imaging
8, 731, (HSC) Medicine - Alternative
9, 1828, (HSC) Medicine - Obstetrics/Gynecology
10, 3823, (HSC) Medicine - Oncology
11, 11837, (BLS) Biology - Cellular
12, 2882, (HSC) Medicine - Pharmacology
13, 1197, (HSC) Medicine - Pulmonology
14, 994, (HSC) Medicine - Anesthesia/Anaesthesia
15, 2559, (HSC) Medicine - Surgery
16, 4830, (BLS) Biology - Molecular
17, 1487, (HSC) Medicine - Orthopedics
18, 1562, (HSC) Medicine - Infectious Disease
19, 563, (HSC) Medicine - Internal
20, 977, (SOC) Communications
21, 1420, (SOC) Sports and Recreation
22, 8780, (B/T) Computer Science
23, 1289, (PHY) Physics
24, 5330, (B/T) Data Science
25, 1451, (HSC) Medicine - General
26, 2779, (PHY) Materials Science
27, 1402, 

## nieuwe kolommen maken
Hierna wou ik uitzoeken hoe ik een nieuwe kolom kon maken op basis van de data in twee andere kollommen hier voor heb ik de onderstaande code geschreven. Hiervoor heb ik de tijd in dagen uitgerekend tussen de orginele publicatie datum en de terugtrek datum.

Mijn laptop liep hier tegen geheugen problemen aan hierom ben ik de data set per chunks gaan inladen. Achteraf was opnieuw opstarten waarschijnlijk sneller en handiger geweest aangezien dit het kortdurend maar mijn probleem fixte.

In [ ]:
def get_chunked_csv(source, index, array, chunksize=100_000):
	df = pd.read_csv(source, index_col=index, usecols=array, chunksize=chunksize)
	return df

def add_counted_dates(df):
	processed_chunks = []
	for chunk in df:
		chunk['OriginalPaperDate'] = pd.to_datetime(chunk['OriginalPaperDate'], errors='coerce')
		chunk['RetractionDate'] = pd.to_datetime(chunk['RetractionDate'], errors='coerce')
		chunk['daysinbetween'] = chunk['RetractionDate'] - chunk['OriginalPaperDate']

		processed_chunks.append(chunk)
		out = pd.concat(processed_chunks, ignore_index=True)
		out.index.name = "ID"
	return out

def save_days_inbetween():
	df = get_chunked_csv(sourcePath, 'Record ID', ['Record ID', 'RetractionDate', 'OriginalPaperDate'], 100_000)
	data = add_counted_dates(df)
	save_csv(data, dataPath + "dates.csv")
	print_dataframe(data)

save_days_inbetween()

Na het bovenstaande ben ik mijn code gaan herstructureren door het in 3 verschillende bestanden te laden namelijk

root
| - genlib.py
| - index.py
| - lib.py
| - data
--| - source.csv


